More Word Vectors
==========
Ref: https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-3-more-fun-with-word-vectors

Numeric Representations of Words
--------------
The Word2Vec model consists of a feature vector for each word in the vocabulary, stored in a numpy array called syn0:

In [1]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model=Word2Vec.load("300features_40minwords_10context")

In [2]:
type(model.syn0)

numpy.ndarray

In [3]:
model.syn0.shape

(16731, 300)

Setting the minimum word count to 40 gave us a total vocaabulary of 16731 words with 300 features apiece. 

In [4]:
# Access individual word vectors
model["flower"]

array([ 0.02134321, -0.00057929, -0.08828005,  0.02148496,  0.00455311,
        0.04645799,  0.13440727, -0.01924068,  0.0250187 ,  0.00716184,
       -0.07705119, -0.07269628, -0.06724421,  0.10513379, -0.01986868,
        0.08252697,  0.01312089, -0.11145967,  0.05409377, -0.06306233,
       -0.16053131,  0.04976825,  0.11018376,  0.12966742, -0.0786479 ,
        0.03784633, -0.02172615, -0.09120998, -0.01007708,  0.00381264,
       -0.0415653 , -0.0004102 , -0.0101001 ,  0.03045666, -0.08472608,
       -0.01462056,  0.03637496,  0.0024175 ,  0.10872315, -0.01177609,
       -0.05104599,  0.01574708, -0.05566194, -0.05069996,  0.00799315,
       -0.02671401, -0.00299433, -0.03807583,  0.0181754 ,  0.14843573,
        0.00807173,  0.00933594,  0.07225905,  0.04447188, -0.05496529,
        0.01974759, -0.02049986,  0.01183401, -0.09530117,  0.01099196,
       -0.07633688,  0.02359863,  0.04380448,  0.00455082, -0.10783515,
        0.07563701, -0.02231596,  0.05592882,  0.06040867,  0.00

From Words to Paragraphs, Attempt 1: Vector Averaging
------------------
Fisrt, we need to take individual word vectors and transform them into a feature set that is the same length for every review. Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method is to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise). 

In [5]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given paragraph
    # Pre-initialize an empty numpy array (for speed)
    featureVec= np.zeros((num_features),dtype="float32")
    nwords=0.
    # Index2word is a list that contains the names of the words in the model's vocabulary.
    # Convert it to a set, for speed
    index2word_set=set(model.index2word)
    # Loop over each word in the review and, if it is in the model's vocabulary,
    # add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords+=1
            featureVec=np.add(featureVec,model[word])
    # Divide the result by the number of words to get the average
    featureVec= np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews,model,num_features):
    # Given a set of reviews (each one is a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array
    # Initialize a counter
    counter=0.
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs= np.zeros((len(reviews),num_features),dtype="float32")
    # Loop through the reviews
    for review in reviews:
        # Print a status message every 1000th review
        if counter%1000.==0.:
            print "Review %d of %d"%(counter,len(reviews))
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter]= makeFeatureVec(review, model, num_features)
        # Increment the counter
        counter+=1.
    return reviewFeatureVecs

**Recall: Word_vectors_p2**

In [6]:
import pandas as pd

# Read data from files
train=pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t",quoting=3)
test=pd.read_csv("testData.tsv", header=0, delimiter="\t",quoting=3)
unlabeled_train=pd.read_csv("unlabeledTrainData.tsv",header=0,delimiter="\t",quoting=3)

# Import various modules for string cleaning 
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review,remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.
    # 1. Remove HTML
    review_text=BeautifulSoup(review,"html").get_text()
    # 2. Remove non-letters, keep numbers
    review_text=re.sub("[^a-zA-Z0-9]"," ",review_text)
    # 3. Convert words in lower case and split them
    words=review_text.lower().split()
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops=set(stopwords.words("english"))
        words=[w for w in words if not w in stops]
    # 5. return a list of words
    return(words)


**Back to Word_vectors_p3**
Now we can call these functions to create average vectors for each paragraph. 

In [7]:
# Calculate the average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word removal.

num_features=300

clean_train_reviews=[]
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review,remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews,model,num_features)

print "Creating average feature vecs for test reviews"
clean_test_reviews=[]
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews,model,num_features)

/Users/anw4/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Review 0 of 25000


/Users/anw4/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:34: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

Next, use the average paragraph vectors to train a random forest. Note that, as in part 1, we can only use the labeled training reviews to train the model.

In [8]:
# Fit a random forest to the trainig data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=100)

print "Fitting a random forest to labeled training data..."
forest=forest.fit(trainDataVecs,train["sentiment"])

# Test and extract results
result= forest.predict(testDataVecs)

# Write the test results
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv",index=False,quoting=3)

Fitting a random forest to labeled training data...


**Need to evaluate: we found that this produced results much better than chance, but underperformed Bag of Words by a few percentage points....**

Since the element-wise average of the vectors didn't produce spectacular results, perhaps we could do it in a more intelligent way? A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract **tf-idf** weights in Python is by using scikit-learn's **TfidfVectorizer**, which has an interface similar to the CountVectorizer that we used in Part 1. However, weighting our word vectors in this way has no substantial improvement in performance.

From Words to Paragraphs, Attempt 2: Clustering
------------------
Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as **vector quantization.** We can find the centers of the word clusters using K-Means (slow). 

In [9]:
from sklearn.cluster import KMeans
import time

start=time.time() 

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0]/5

# Initialize a k-means object and use it to extract centroids
kmeans_clustering= KMeans(n_clusters=num_clusters)
idx= kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end= time.time()
elapsed = end-start
print "Time taken for K Means clustering: ",elapsed," seconds."

Time taken for K Means clustering:  1542.80337405  seconds.


In [11]:
# Create a Word/Index dictionary, mapping each vocabulary word to a cluster number
word_centroid_map= dict(zip(model.index2word,idx))

In [20]:
# For the first 10 clusters
for cluster in xrange(0,10):
    # Print the cluster number
    print "\nCluster %d" %cluster
    # Find all of the words for that cluster number, and print them out
    words=[]
    for i in xrange(0,len(word_centroid_map.values())):
        if (word_centroid_map.values()[i]==cluster):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'supermarket', u'cafe', u'shops', u'diner', u'caf', u'pub', u'gym', u'bar', u'shop', u'restaurant', u'bookstore', u'factory']

Cluster 1
[u'leary', u'magnum', u'conway']

Cluster 2
[u'luzhin', u'patriarch', u'jeb', u'unjustly', u'troublesome', u'proposition', u'eternally', u'gypo', u'embittered', u'impatient', u'withdrawn']

Cluster 3
[u'revue']

Cluster 4
[u'leaps', u'looming']

Cluster 5
[u'troubles', u'scars', u'fears', u'consequences', u'senses', u'deepest', u'sanity']

Cluster 6
[u'void', u'innovation']

Cluster 7
[u'seduced', u'befriended', u'traumatized', u'teased', u'personified', u'rejected', u'mistreated', u'thwarted', u'pursued', u'recruited', u'victimized', u'humiliated', u'bullied', u'harassed', u'framed', u'terrorized', u'ridiculed', u'tormented', u'betrayed']

Cluster 8
[u'1958', u'1957', u'1956', u'1959', u'1961', u'1912', u'1965', u'1964', u'1962', u'1944', u'1945', u'1916']

Cluster 9
[u'slash', u'stalk', u'toolbox']


In [21]:
def create_bag_of_centroids(wordlist,word_centroid_map):
    # The number of clusters is equal to the highest cluster index in the word/centroid map
    num_centroids=max(word_centroid_map.values())+1
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids=np.zeros(num_centroids,dtype="float32")
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count by one
    for word in wordlist:
        if word in word_centroid_map:
            index=word_centroid_map[word]
            bag_of_centroids[index]+=1
    return bag_of_centroids


The function above will give us a numpy array for each review, each with a number of features equal to the number of clusters. Finally, we create bags of centroids for the training and testing sets, then train a random forest.

In [22]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids=np.zeros((train["review"].size,num_clusters),dtype="float32")

# Transform the training set reviews into bags of centroids
counter =0 
for review in clean_train_reviews:
    train_centroids[counter]=create_bag_of_centroids(review,word_centroid_map)
    counter+=1
    
# Repeat for test reviews
test_centroids=np.zeros((test["review"].size,num_clusters),dtype="float32")
counter=0
for review in clean_test_reviews:
    test_centroids[counter]=create_bag_of_centroids(review,word_centroid_map)
    counter+=1

In [25]:
# Fit a random forest and extract predictions
forest=RandomForestClassifier(n_estimators=100)

# Fitting the forest 
print "Fitting a random forest to labeled training data..."
forest=forest.fit(train_centroids,train["sentiment"])
result=forest.predict(test_centroids)

# Write the test results
ouptput=pd.DataFrame(data={"id":test["id"],"sentiment":result})
output.to_csv("Bag_of_Centroids.csv",index=False,quoting=3)

Fitting a random forest to labeled training data...


**Need to evaluate: The code above gives about the same (or slightly worse) results compared to the Bag of Words in Part 1.**